In [12]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
client_id = "Qph1X9woDtOVd8rFK17U"
client_secret = "ZKaKFR3jzg"
encText = urllib.parse.quote("부동산")
url = "https://openapi.naver.com/v1/search/news.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/news.json?query=부동산&display=10&start=1&sort=sim"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Sun, 24 Nov 2024 15:03:20 +0900",
	"total":4793756,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"&quot;경원선 지하화를··· 교통 정체 해소·지역 발전&quot;",
			"originallink":"https:\/\/www.siminilbo.co.kr\/news\/newsview.php?ncode=1160287233918117",
			"link":"https:\/\/www.siminilbo.co.kr\/news\/newsview.php?ncode=1160287233918117",
			"description":"지하화를 실현할 경우 기존 지상부 공간은 공원, 문화시설, 복합시설 등으로 재조성돼 지역 주민들의 삶의 이 향상됨과 동시에 도심 생태계 복원, 지역 경제 활성화와 <b>부동산<\/b> 가치 상승 등 다양한 경제적 효과도 따라 올 것... ",
			"pubDate":"Sun, 24 Nov 2024 15:02:00 +0900"
		},
		{
			"title":"&quot;옛 명성 포에버&quot;…한샘 '정상 탈환' 노린다",
			"originallink":"http:\/\/www.enewstoday.co.kr\/news\/articleView.html?idxno=2204039",
			"link":"http:\/\/www.enewstoday.co.kr\/news\/articleView.html?idxno=2204039",
			"description":"한샘은 <b>부동산<\/b> 경기 회복 국면을 대비해 이들 핵심 상품과 프로모션의 마케팅 고도화와 고객 침투율 제고 등 마케팅 역량을 집중해 나가고 있다. 특히 한샘의 주력 캠페인인 '쌤페스타'는 매 회차 주문액 경신을 이어가며... ",
			"pubDate":"Sun, 24 Nov 2024 15:00:00 +0900"
		},
		{
			"title":"與 서

In [43]:
import requests
from bs4 import BeautifulSoup
import re

# 요청할 URL
url = "https://news.naver.com/section/template/SECTION_ARTICLE_LIST?sid=101&sid2=&cluid=&pageNo=&date=&next=20241124110000"

# HTTP GET 요청을 보내고 응답을 받음
response = requests.get(url)

# 요청이 성공적으로 이루어졌는지 확인 (상태 코드가 200인 경우)
if response.status_code == 200:
    # HTML 내용을 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # "https://n.news.naver.com/mnews/article/{id}/{id}" 형식의 URL만 필터링 (id는 숫자와 문자가 섞인 형식)
    href_list = list({
        a.get('href').replace("\\", "").replace("\"", "")
        for a in soup.find_all('a', href=True)
        if re.search(r'https://n\.news\.naver\.com/mnews/article/\d{3}/\d+', a.get('href'))
    })
    
    # URL 리스트 정렬
    href_list.sort()

    next_div = soup.find_all('div', {'data-cursor-name': 'next'})
    print(next_div)
    
    # 결과 출력
    print(len(href_list))
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

[]
36


In [69]:
import requests
import json
import time
from bs4 import BeautifulSoup

# 요청할 URL
url = "https://news.naver.com/section/template/SECTION_ARTICLE_LIST?sid=101&sid2=&cluid=&pageNo=&date=&next=20241124110000"

# HTTP GET 요청을 보내고 응답을 받음
response = requests.get(url)

# 요청이 성공적으로 이루어졌는지 확인 (상태 코드가 200인 경우)
if response.status_code == 200:
    # JSON 데이터 파싱
    json_data = response.json()

    # renderedComponent 내에서 SECTION_ARTICLE_LIST 추출
    try:
        rendered_component = json_data['renderedComponent']
        section_article_list_html = rendered_component.get('SECTION_ARTICLE_LIST', '')

        # SECTION_ARTICLE_LIST의 값이 HTML이면 BeautifulSoup으로 파싱
        if section_article_list_html:
            soup = BeautifulSoup(section_article_list_html, 'html.parser')
            
            next_div = soup.find('div', {'data-cursor-name': 'next'})
            # print(next_div.get("data-cursor"))
            
            # "https://n.news.naver.com/mnews/article/{id}/{id}" 형식의 URL만 필터링 (id는 숫자와 문자가 섞인 형식)
            href_list = list({
                a.get('href').replace("\\", "").replace("\"", "")
                for a in soup.find_all('a', href=True)
                if re.search(r'https://n\.news\.naver\.com/mnews/article/\d{3}/\d+', a.get('href'))
            })
            
            # URL 리스트 정렬
            href_list.sort()

            article_data_list = []
            for url in href_list:
                article_data_list.append(extract_article_data(url))
                time.sleep(3)
            print(len(article_data_list))
            print(article_data_list[-1])
        else:
            print("SECTION_ARTICLE_LIST 내용이 없습니다.")
    except KeyError as e:
        print(f"'{e}' 키를 찾을 수 없습니다.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    

36
None


In [67]:
import requests
from bs4 import BeautifulSoup

# URL 설정
url = "https://n.news.naver.com/mnews/article/001/0015063501"

# constant
TITLE = 'title'
CREATED_AT = 'created_at'
UPDATED_AT = 'updated_at'
CONTENT_ORIGIN = 'content_origin'
CONTENT_TEXT = 'content_text'

def init_article_data():
    return {
        TITLE: None,
        CREATED_AT: None,
        UPDATED_AT: None,
        CONTENT_ORIGIN: None,
        CONTENT_TEXT: None
    }

def extract_article_data(url): 
    article_data = init_article_data()
    
    # HTTP 요청 보내기
    response = requests.get(url)
    
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 기사 제목 추출
    title_node = soup.find('h2', {'id': 'title_area'})
    article_data[TITLE] = title_node.text.strip() if title_node else None
    
    # 입력일자 추출
    created_at_node = soup.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'})
    article_data[CREATED_AT] = created_at_node.get('data-date-time') if created_at_node else None
    
    
    # 수정일자 추출
    updated_at_node = soup.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME'})
    article_data[UPDATED_AT] = updated_at_node.get('data-modify-date-time') if updated_at_node else None
    
    # 기사 내용 추출
    content_node = soup.find('article', {'id': 'dic_area'})
    article_data[CONTENT_TEXT] = content_node.text.strip() if content_node else None
    article_data[CONTENT_ORIGIN] = content_node if content_node else None

# 결과 출력
# print("기사 제목:", title)
# print("입력일자:", input_date)
# print("수정일자:", edit_date)
# print("기사 내용:", content_origin)

기사 제목: 살빼는 약 '위고비' 인기지만…"5명중 1명, 체중감량 효과 없어"
입력일자: 2024-11-24 10:14:40
수정일자: 2024-11-24 10:15:12
기사 내용: <article class="go_trans _article_content" id="dic_area">
<strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">임상시험서 10∼15% '비반응자', 본격 사용되니 20% 수준 상승</strong><br/><br/><span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img alt="'위고비 드디어 입고'
(서울=연합뉴스) 서대연 기자 = 2024년 10월 17일 오후 서울 시내 한 약국에 비만치료제 위고비 입고 안내문이 붙어있다.
    위고비는 혈당 조절에 중요한 인슐린 분비를 촉진해 식욕 " class="_LAZY_LOADING _LAZY_LOADING_INIT_HIDE" data-src="https://imgnews.pstatic.net/image/001/2024/11/24/PYH2024101716910001300_P4_20241124101512433.jpg?type=w860" id="img1" style="display: none;">
</img></div>
</div><em class="img_desc">'위고비 드디어 입고'<br/>(서울=연합뉴스) 서대연 기자 = 2024년 10월 17일 오후 서울 시내 한 약국에 비만치료제 위고비 입고 안내문이 붙어있다.<br/>    위고비는 혈당 조절에 중요한 인슐린 분비를 촉진해 식욕

In [60]:
def find_node(target="", tag='div', attribute={}):
    target.find(tag, attribute)